In [1]:
!pip uninstall torch -y
!pip install http://download.pytorch.org/whl/cu80/torch-0.3.1-cp36-cp36m-linux_x86_64.whl  && pip install torchvision

Uninstalling torch-0.3.1:
  Successfully uninstalled torch-0.3.1
    100% |████████████████████████████████| 496.4MB 41.0MB/s 


In [2]:
!pip install git+https://github.com/fastai/fastai@2e1ccb58121dc648751e2109fc0fbf6925aa8887
!apt update && apt install -y libsm6 libxext6

  Cloning https://github.com/fastai/fastai (to revision 2e1ccb58121dc648751e2109fc0fbf6925aa8887) to /tmp/pip-req-build-81f76pao
  Running setup.py bdist_wheel for fastai ... - \ | done
  Stored in directory: /tmp/pip-ephem-wheel-cache-b7jwv1qp/wheels/25/e9/9b/223e47f59c39f0bd393733db0f7efe8ec6f078d29c32198f9f
Successfully built fastai
Hit:1 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:2 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu artful-security InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
All packages are up to date.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [3]:
!pip install feedparser

In [6]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
!echo "Hard drive info\n==========================="
!df -H /
!echo "\nRAM info\n==========================="
!free -h
!echo "\nVM User info\n==========================="
!whoami

Hard drive info\n===========================
Filesystem      Size  Used Avail Use% Mounted on
overlay         385G   13G  353G   4% /
\nRAM info\n===========================
              total        used        free      shared  buff/cache   available
Mem:            12G        929M        735M        251M         11G         11G
Swap:            0B          0B          0B
\nVM User info\n===========================
root


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import random

In [0]:
bs,bptt = 64,70

## Language modeling

### Data

In [0]:
import os, requests, time
# feedparser isn't a fastai dependency so you may need to install it.
import feedparser
import pandas as pd


class GetArXiv(object):
    def __init__(self, pickle_path, categories=list()):
        """
        :param pickle_path (str): path to pickle data file to save/load
        :param pickle_name (str): file name to save pickle to path
        :param categories (list): arXiv categories to query
        """
        if os.path.isdir(pickle_path):
            pickle_path = f"{pickle_path}{'' if pickle_path[-1] == '/' else '/'}all_arxiv.pkl"
        if len(categories) < 1:
            categories = ['cs*', 'cond-mat.dis-nn', 'q-bio.NC', 'stat.CO', 'stat.ML']
        # categories += ['cs.CV', 'cs.AI', 'cs.LG', 'cs.CL']

        self.categories = categories
        self.pickle_path = pickle_path
        self.base_url = 'http://export.arxiv.org/api/query'

    @staticmethod
    def build_qs(categories):
        """Build query string from categories"""
        return '+OR+'.join(['cat:'+c for c in categories])

    @staticmethod
    def get_entry_dict(entry):
        """Return a dictionary with the items we want from a feedparser entry"""
        try:
            return dict(title=entry['title'], authors=[a['name'] for a in entry['authors']],
                        published=pd.Timestamp(entry['published']), summary=entry['summary'],
                        link=entry['link'], category=entry['category'])
        except KeyError:
            print('Missing keys in row: {}'.format(entry))
            return None

    @staticmethod
    def strip_version(link):
        """Strip version number from arXiv paper link"""
        return link[:-2]

    def fetch_updated_data(self, max_retry=5, pg_offset=0, pg_size=1000, wait_time=15):
        """
        Get new papers from arXiv server
        :param max_retry: max number of time to retry request
        :param pg_offset: number of pages to offset
        :param pg_size: num abstracts to fetch per request
        :param wait_time: num seconds to wait between requests
        """
        i, retry = pg_offset, 0
        df = pd.DataFrame()
        past_links = []
        if os.path.isfile(self.pickle_path):
            df = pd.read_pickle(self.pickle_path)
            df.reset_index()
        if len(df) > 0: past_links = df.link.apply(self.strip_version)

        while True:
            params = dict(search_query=self.build_qs(self.categories),
                          sortBy='submittedDate', start=pg_size*i, max_results=pg_size)
            response = requests.get(self.base_url, params='&'.join([f'{k}={v}' for k, v in params.items()]))
            entries = feedparser.parse(response.text).entries
            if len(entries) < 1:
                if retry < max_retry:
                    retry += 1
                    time.sleep(wait_time)
                    continue
                break

            results_df = pd.DataFrame([self.get_entry_dict(e) for e in entries])
            max_date = results_df.published.max().date()
            new_links = ~results_df.link.apply(self.strip_version).isin(past_links)
            print(f'{i}. Fetched {len(results_df)} abstracts published {max_date} and earlier')
            if not new_links.any():
                break

            df = pd.concat((df, results_df.loc[new_links]), ignore_index=True)
            i += 1
            retry = 0
            time.sleep(wait_time)

        print(f'Downloaded {len(df)-len(past_links)} new abstracts')
        df.sort_values('published', ascending=False).groupby('link').first().reset_index()
        df.to_pickle(self.pickle_path)
        return df

    @classmethod
    def load(cls, pickle_path):
        """Load data from pickle and remove duplicates"""
        return pd.read_pickle(cls(pickle_path).pickle_path)

    @classmethod
    def update(cls, pickle_path, categories=list(), **kwargs):
        """
        Update arXiv data pickle with the latest abstracts
        """
        cls(pickle_path, categories).fetch_updated_data(**kwargs)
        return True

In [13]:
!mkdir data
!cd data && mkdir arxiv
!ls
!cd data && ls

data  sample_data
arxiv


In [15]:
from google.colab import files
uploaded = files.upload()

Saving arxiv.csv to arxiv.csv


In [0]:
!mv arxiv.csv data/arxiv/

In [18]:
!cd data/arxiv && ls

arxiv.csv


In [19]:
PATH='data/arxiv/'

ALL_ARXIV = f'{PATH}all_arxiv.pkl'

# all_arxiv.pkl: if arxiv hasn't been downloaded yet, it'll take some time to get it - go get some coffee
if not os.path.exists(ALL_ARXIV): GetArXiv.update(ALL_ARXIV)

# arxiv.csv: see dl1/nlp-arxiv.ipynb to get this one
df_mb = pd.read_csv(f'{PATH}arxiv.csv')
df_all = pd.read_pickle(ALL_ARXIV)

0. Fetched 1000 abstracts published 2018-10-04 and earlier
1. Fetched 1000 abstracts published 2018-09-26 and earlier
2. Fetched 1000 abstracts published 2018-09-18 and earlier
3. Fetched 1000 abstracts published 2018-09-10 and earlier
4. Fetched 1000 abstracts published 2018-09-03 and earlier
5. Fetched 1000 abstracts published 2018-08-25 and earlier
6. Fetched 1000 abstracts published 2018-08-14 and earlier
7. Fetched 1000 abstracts published 2018-08-04 and earlier
Downloaded 8000 new abstracts


In [20]:
def get_txt(df):
    return '<CAT> ' + df.category.str.replace(r'[\.\-]','') + ' <SUMM> ' + df.summary + ' <TITLE> ' + df.title
df_mb['txt'] = get_txt(df_mb)
df_all['txt'] = get_txt(df_all)
n=len(df_all); n

8000

In [0]:
os.makedirs(f'{PATH}trn/yes', exist_ok=True)
os.makedirs(f'{PATH}val/yes', exist_ok=True)
os.makedirs(f'{PATH}trn/no', exist_ok=True)
os.makedirs(f'{PATH}val/no', exist_ok=True)
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)

In [0]:
for (i,(_,r)) in enumerate(df_all.iterrows()):
    dset = 'trn' if random.random()>0.1 else 'val'
    open(f'{PATH}all/{dset}/{i}.txt', 'w').write(r['txt'])

In [0]:
for (i,(_,r)) in enumerate(df_mb.iterrows()):
    lbl = 'yes' if r.tweeted else 'no'
    dset = 'trn' if random.random()>0.1 else 'val'
    open(f'{PATH}{dset}/{lbl}/{i}.txt', 'w').write(r['txt'])

In [25]:
!python -m spacy download en 

    100% |████████████████████████████████| 37.4MB 1.1MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
from spacy.symbols import ORTH

# install the 'en' model if the next line of code fails by running:
#python -m spacy download en              # default English model (~50MB)
#python -m spacy download en_core_web_md  # larger English model (~1GB)
my_tok = spacy.load('en')

my_tok.tokenizer.add_special_case('<SUMM>', [{ORTH: '<SUMM>'}])
my_tok.tokenizer.add_special_case('<CAT>', [{ORTH: '<CAT>'}])
my_tok.tokenizer.add_special_case('<TITLE>', [{ORTH: '<TITLE>'}])
my_tok.tokenizer.add_special_case('<BR />', [{ORTH: '<BR />'}])
my_tok.tokenizer.add_special_case('<BR>', [{ORTH: '<BR>'}])

def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

In [0]:
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
FILES = dict(train='trn', validation='val', test='val')
md = LanguageModelData.from_text_files(f'{PATH}all/', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [45]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(326, 7420, 1, 1465781)

In [46]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'of', '-', 'and', 'a', 'to', 'in', 'we']

In [47]:
' '.join(md.trn_ds[0].text[:150])

'<cat> condmatmtrlsci <summ> we present example applications of an approach to first - principles calculations of vibrational properties of materials implemented within the exabyte.io platform . we deploy models based on the density functional perturbation theory to extract the phonon dispersion relations and densities of states for an example set of 35 samples and find the results to be in agreement with prior similar calculations . we construct modeling workflows that are both accessible , accurate , and efficient with respect to the human time involved . this is achieved through efficient parallelization of the tasks for the individual vibrational modes . we report achieved speedups in the 10 - 100 range , approximately , and maximum attainable speedups in the 30 - 300 range , correspondingly . we analyze the execution times on the current up - to - date computational infrastructure centrally available from a public cloud provider'

### Train

In [0]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [0]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [54]:
learner.fit(3e-3, 1, wds=1e-6)

epoch      trn_loss   val_loss   
    0      5.235505   5.017305  



[array([5.01731])]

In [55]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      4.837221   4.738758  
    1      4.660362   4.521287  
    2      4.488184   4.440642  
    3      4.467755   4.360261  
    4      4.309736   4.267489  
    5      4.194652   4.219086  
    6      4.134557   4.211936  



[array([4.21194])]

In [0]:
learner.save_encoder('adam2_enc')

In [0]:
learner.fit(3e-3, 10, wds=1e-6, cycle_len=5, cycle_save_name='adam3_10')

A Jupyter Widget

[ 0.       3.89388  3.76575]                                  
[ 1.       3.82548  3.71875]                                  
[ 2.       3.76471  3.66974]                                  
[ 3.       3.71713  3.63861]                                  
[ 4.       3.67534  3.62983]                                  
[ 5.       3.83938  3.71551]                                  
[ 6.       3.78093  3.68056]                                  
[ 7.       3.72828  3.63638]                                  
[ 8.       3.66743  3.60355]                                  
[ 9.       3.65793  3.59448]                                  
[ 10.        3.80545   3.68213]                               
[ 11.        3.75299   3.65219]                               
[ 12.        3.7057    3.61324]                               
[ 13.        3.63348   3.58048]                               
[ 14.        3.62304   3.57257]                               
[ 15.        3.78656   3.66324]                        

In [0]:
learner.save_encoder('adam3_10_enc')

In [0]:
learner.fit(3e-3, 8, wds=1e-6, cycle_len=10, cycle_save_name='adam3_5')

A Jupyter Widget

[ 0.       3.70587  3.61666]                                  
[ 1.       3.71738  3.61174]                                  
[ 2.       3.68606  3.59661]                                  
[ 3.       3.65407  3.5742 ]                                  
[ 4.       3.62901  3.55795]                                  
[ 5.       3.59921  3.53632]                                  
[ 6.       3.58401  3.52149]                                  
[ 7.       3.55126  3.50797]                                  
[ 8.       3.52965  3.50178]                                  
[ 9.       3.52336  3.49997]                                  
[ 10.        3.7109    3.60817]                               
[ 11.        3.69879   3.60047]                               
[ 12.        3.6735    3.58623]                               
[ 13.        3.64365   3.56568]                               
[ 14.        3.6099    3.54776]                               
[ 15.        3.58244   3.52829]                        

In [0]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

A Jupyter Widget

[ 0.       3.47841  3.4751 ]                                  
[ 1.       3.69717  3.57883]                                  
[ 2.       3.68267  3.57793]                                  
[ 3.       3.66797  3.57299]                                  
[ 4.       3.66805  3.56847]                                  
[ 5.       3.63489  3.56238]                                  
[ 6.       3.62479  3.54928]                                  
[ 7.       3.60663  3.53879]                                  
[ 8.       3.59124  3.53175]                                  
[ 9.       3.58617  3.52009]                                  
[ 10.        3.56924   3.51174]                               
[ 11.        3.5509    3.49974]                               
[ 12.        3.51595   3.49008]                               
[ 13.        3.50939   3.48222]                               
[ 14.        3.48886   3.47952]                               
[ 15.        3.4676    3.47311]                        

In [0]:
learner.save_encoder('adam3_20_enc')

In [0]:
learner.save('adam3_20')

#(TO load the pretrained weights)

In [0]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3


In [52]:
learner

SequentialRNN(
  (0): RNN_Encoder(
    (encoder): Embedding(7420, 200, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(7420, 200, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(200, 500)
      )
      (1): WeightDrop(
        (module): LSTM(500, 500)
      )
      (2): WeightDrop(
        (module): LSTM(500, 200)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=200, out_features=7420, bias=False)
    (dropout): LockedDropout(
    )
  )
)

In [38]:
from google.colab import files
uploaded = files.upload()

Saving imdb_adam3_c1_cl10_cyc_0.h5 to imdb_adam3_c1_cl10_cyc_0.h5
Saving imdb2x_cyc_4.h5 to imdb2x_cyc_4.h5


In [40]:
!cd data/arxiv && mkdir all
!cd data/arxiv/all && mkdir models
! mv imdb_adam3_c1_cl10_cyc_0.h5 data/arxiv/all/models

mkdir: cannot create directory ‘all’: File exists
mkdir: cannot create directory ‘models’: File exists


In [0]:
learner.load('imdb_adam3_c1_cl10_cyc_0')

### Test

In [0]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [0]:
m=learner.model

In [0]:
s="""<CAT> cscv <SUMM> algorithms that"""

In [0]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [0]:
sample_model(m,"<CAT> csni <SUMM> algorithms that")

...use the same network as a single node are not able to 
 achieve the same performance as the traditional network - based routing 
 algorithms . in this paper , we propose a novel routing scheme for routing 
 protocols in wireless networks . the proposed scheme is based ...

In [0]:
sample_model(m,"<CAT> cscv <SUMM> algorithms that")

...use the same data to perform image classification are 
 increasingly being used to improve the performance of image classification 
 algorithms . in this paper , we propose a novel method for image classification 
 using a deep convolutional neural network ( cnn ) . the proposed method is ...

In [0]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> on ")

...the performance of deep learning for image classification <eos> 

In [0]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> on ")

...the performance of wireless networks <eos> 

In [0]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> towards ")

...a new approach to image classification <eos> 

In [0]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> towards ")

...a new approach to the analysis of wireless networks <eos> 

### Sentiment

In [0]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [0]:
class ArxivDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['yes', 'no']:
            fnames = glob(os.path.join(path, label, '*.txt'));
            assert fnames, f"can't find 'yes.txt' or 'no.txt' under {path}/{label}"
            for fname in fnames:
                with open(fname, 'r') as f: text = f.readline()
                examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [0]:
ARX_LABEL = data.Field(sequential=False)
splits = ArxivDataset.splits(TEXT, ARX_LABEL, PATH, train='trn', test='val')

In [0]:
md2 = TextData.from_splits(PATH, splits, bs)

In [0]:
#            dropout=0.3, dropouti=0.4, wdrop=0.3, dropoute=0.05, dropouth=0.2)

In [0]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_at_6(preds,targs):
    precision, recall, _ = precision_recall_curve(targs==2, preds[:,2])
    print(recall[precision>=0.6][0])
    return recall[precision>=0.6][0]

In [0]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.65, wdrop=0.5, dropoute=0.1, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.clip=25.

In [0]:
# this notebook has a mess of some things going under 'all/' others not, so a little hack here
!ln -sf ../all/models/adam3_20_enc.h5 {PATH}models/adam3_20_enc.h5
m3.load_encoder(f'adam3_20_enc')
lrs=np.array([1e-4,1e-3,1e-3,1e-2,3e-2])

In [0]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

A Jupyter Widget

[ 0.       0.47654  0.44322  0.78525]                         



A Jupyter Widget

[ 0.       0.43033  0.40192  0.80087]                        



In [0]:
m3.fit(lrs, 2, metrics=[accuracy], cycle_len=4, cycle_save_name='imdb2')

A Jupyter Widget

[ 0.       0.42236  0.39006  0.8194 ]                        
[ 1.       0.39477  0.37063  0.82086]                        
[ 2.       0.39389  0.37082  0.82449]                        
[ 3.       0.40728  0.36999  0.82195]                        
[ 4.       0.39308  0.3675   0.81977]                        
[ 5.       0.38662  0.36737  0.8234 ]                        
[ 6.       0.39259  0.36512  0.82486]                        
[ 7.       0.38047  0.36538  0.82522]                        



In [0]:
prec_at_6(*m3.predict_with_targs())

0.659305993691


0.65930599369085174

In [0]:
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

A Jupyter Widget

[ 0.       0.38752  0.36351  0.82486]                        
[ 1.       0.38664  0.36123  0.82558]                        
[ 2.       0.3904   0.36098  0.82486]                        
[ 3.       0.37319  0.36144  0.82486]                        
[ 4.       0.38074  0.36334  0.82595]                        
[ 5.       0.36405  0.3594   0.82413]                        
[ 6.       0.38781  0.35914  0.82522]                        
[ 7.       0.37722  0.357    0.82631]                        



In [0]:
prec_at_6(*m3.predict_with_targs())

0.695583596215


0.69558359621451105

#END